# nesso models

## Introduction
This tutorial provides a basic walkthrough of the `nesso models` workflow. To learn about all available `nesso models` features, see [nesso CLI docs](https://nesso_cli.docs.hetzner.dyvenia.lan/).

This project uses the following schema structure:

- bronze schema: `landing`
- intermediate schema: `intermediate`
- marts schema: `marts`
- silver schema prefix: `int`

Remember that to get more information about any `nesso` CLI command, you can always run it with the `--help` flag.

## Glossary
Below is a glossary of some of the terms used in this tutorial.  

- bronze schema: The schema where data is initially ingested.
- intermediate schema: The schema where data is transformed and cleaned.
- marts schema: The user-facing schema, where data is ready for consumption by dashboards, reports, etc.
- silver schema prefix: Prefix that is added to the model name to make it unique, because dbt not allowing models with the same name (even if they're located in different schemas).

## Tutorial

### 1. Project set up


In [ ]:
git clone https://github.com/dyvenia/nesso-cli && \
  cd nesso-cli/examples/nesso_models && \
  source .venv/bin/activate &&
  pip install -r requirements.txt

### 2. Adding source tables
This step consists of two parts:  
- Ingesting tables into the bronze schema  
- Adding the tables into the source tables schema YAML file with `nesso models source add`  

**NOTE**: This step is usually handled by a data engineering team.

### Load some fake data into the demo database

In [ ]:
python create_tutorial_tables.py

### Register the tables as dbt sources
Here, we will make `dbt` aware of the tables we just ingested, so that we can later refer to them with the `{{ source("landing", "my_table") }}` syntax (if you don't know what this is, be sure to complete the [dbt fundamentals](https://courses.getdbt.com/courses/fundamentals) course before proceeding).


**NOTE**: Please ignore the output of the following commands.

In [3]:
nesso models source add account -ni 

Source table 'staging.account' already exists. Skipping...
(.venv) 


In [4]:
nesso models source add contact -ni 

Source table 'staging.contact' already exists. Skipping...
(.venv) 


### 3. Creating base models
Here, we create base models for our source tables.
First, we let `nesso models base_model bootstrap` create an empty SQL file for us (it will create required folders and use proper naming conventions):

In [ ]:
nesso models base_model bootstrap account

In [ ]:
nesso models base_model bootstrap contact

Now we can add our SQL inside the generated files (`models/intermediate/int_account/int_account.sql`, `models/intermediate/int_contact/int_contact.sql`).

#### Bootstrapping the YAML file
Now we can create a template YAML file for our base models:

In [ ]:
nesso models base_model bootstrap-yaml account

In [ ]:
nesso models base_model bootstrap-yaml contact

Now you can also execute tests for your models:

In [ ]:
nesso models test --select account

In [ ]:
nesso models test -s contact

<div class="alert alert-block alert-info"> <b>Tip:</b> `-s` is shorthand for `--select` </div>

### 4. Creating models
#### Bootstrapping the SQL file
First, we let `nesso models` create an empty SQL file for us (it will create required folders and use proper naming conventions):

In [ ]:
nesso models model bootstrap account --subdir sales

Now we can add our SQL inside the generated file (`models/marts/sales/account/account.sql`).

#### Bootstrapping the YAML file
Now we can create a template YAML file for our model:

In [ ]:
nesso models model bootstrap-yaml account

We can now add any required metadata and tests to the model YAML. 

See the [models user guide](https://nesso_cli.docs.hetzner.dyvenia.lan/user_guide/models) for more information on how to add metadata and tests.

#### Testing the model

To run tests for this model, execute the following command:

In [ ]:
nesso models test -s account